# Customer Data Analysis

# IMPORT LIBRARIES 

In [1]:
import pandas as pd 
import numpy as np 
import matplotlib as plt
import pyodbc
from datetime import datetime

# IMPORT DATA and CONNECTING TO SQL

In [9]:
conn = pyodbc.connect(
    'DRIVER={SQL Server};'
    'SERVER=.;'
    'DATABASE=CUSTOMERDB;'
    'UID=Nssr;'
    'PWD=abcd@123'
)

# DATA LOADING

In [11]:
#WRITING SQL QUERIES 
query = "SELECT * from Customer_Data"
query2 = "SELECT * from interaction_data"
query3 = "SELECT * from Order_Data"
query4 = "SELECT * from Transaction_Data"
# DATA FRAME LOADING USING PANDAS AND SQL QUERIES 
customer_data = pd.read_sql(query, conn)
interaction_data = pd.read_sql(query2, conn)
order_data = pd.read_sql(query3, conn)
transaction_data  = pd.read_sql(query4, conn)
conn.close()

C:\Users\user\AppData\Local\Temp\ipykernel_20708\1168789643.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  customer_data = pd.read_sql(query, conn)
C:\Users\user\AppData\Local\Temp\ipykernel_20708\1168789643.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  interaction_data = pd.read_sql(query2, conn)
C:\Users\user\AppData\Local\Temp\ipykernel_20708\1168789643.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  order_data = pd.read_sql(query3, conn)
C:\Users\user\AppData\Local\Temp\ipykernel_20708\1168789643.py:11: 

# DATA ANALYZING

# customer and interaction

In [15]:

customer_data['Join_Date'] = pd.to_datetime(customer_data['Join_Date'], errors='coerce')
customer_data['Date_Assigned'] = pd.to_datetime(customer_data['Date_Assigned'], errors='coerce')
customer_data['Birthdate'] = pd.to_datetime(customer_data['Birthdate'], errors='coerce')
# 1. Age Analysis of Customers
# Correct way to calculate age in years by dividing by 365.25 to account for leap years
customer_data['Age'] = (pd.Timestamp.now() - customer_data['Birthdate']).dt.days / 365.25
age_analysis = customer_data['Age'].describe()
# Print the age analysis
print("Age Analysis of Customers:")
print(age_analysis)



Age Analysis of Customers:
count    2999.000000
mean       54.675810
std        21.171440
min        18.080767
25%        36.258727
50%        54.266940
75%        73.141684
max        91.063655
Name: Age, dtype: float64


In [16]:
# 2. Interaction Frequency by Customer Segment
# Merge customer data with interaction data on Customer_ID
merged_data = pd.merge(interaction_data, customer_data, on='Customer_ID', how='inner')

# Group interactions by customer segment
interaction_by_segment = merged_data.groupby('Segment')['Interaction_ID'].count()

# Print the interaction frequency by segment
print("\nInteraction Frequency by Segment:")
print(interaction_by_segment)


Interaction Frequency by Segment:
Segment
Premium    1030
Regular     987
VIP         982
Name: Interaction_ID, dtype: int64


In [17]:
# 3. Interaction Types Distribution Across Customer Segments
interaction_type_by_segment = merged_data.groupby(['Segment', 'Interaction_Type']).size().unstack(fill_value=0)

# Print the interaction types by segment
print("\nInteraction Types by Segment:")
print(interaction_type_by_segment)



Interaction Types by Segment:
Interaction_Type  Email  In-Person Meeting  Phone Call  \
Segment                                                  
Premium             262                248         254   
Regular             258                257         243   
VIP                 235                280         242   

Interaction_Type  Social Media Interaction  
Segment                                     
Premium                                266  
Regular                                229  
VIP                                    225  


# customer and orders

In [19]:
# Parse date columns to allow for date-based operations
customer_data['Join_Date'] = pd.to_datetime(customer_data['Join_Date'], errors='coerce')
customer_data['Date_Assigned'] = pd.to_datetime(customer_data['Date_Assigned'], errors='coerce')
customer_data['Birthdate'] = pd.to_datetime(customer_data['Birthdate'], errors='coerce')
order_data['Order_Date'] = pd.to_datetime(order_data['Order_Date'], errors='coerce')

# 1. Basic summary statistics for both datasets
customer_summary = customer_data.describe(include='all')
order_summary = order_data.describe()

# Print the summaries
print("Customer Data Summary:")
print(customer_summary)

print("\nOrder Data Summary:")
print(order_summary)

# 2. Analyzing order trends over time (grouped by month)
order_trends = order_data.groupby(order_data['Order_Date'].dt.to_period('M')).size()

# Print order trends over time
print("\nOrder Trends Over Time:")
print(order_trends)
# 3. Analyzing number of orders per customer segment
# Merging customer and order data
merged_data = pd.merge(order_data, customer_data, on='Customer_ID', how='inner')

# Grouping by customer segment and counting orders
orders_by_segment = merged_data.groupby('Segment')['Order_ID'].count()

# Print the number of orders by customer segment
print("\nNumber of Orders by Customer Segment:")
print(orders_by_segment)

Customer Data Summary:
        Customer_ID First_Name Last_Name            Email              Phone  \
count   2999.000000       2999      2999             2999               2999   
unique          NaN        540       831             2993               2999   
top             NaN    Michael     Smith  jgill@gmail.com  (442)749-3583x632   
freq            NaN         79        72                2                  1   
mean    1500.000000        NaN       NaN              NaN                NaN   
min        1.000000        NaN       NaN              NaN                NaN   
25%      750.500000        NaN       NaN              NaN                NaN   
50%     1500.000000        NaN       NaN              NaN                NaN   
75%     2249.500000        NaN       NaN              NaN                NaN   
max     2999.000000        NaN       NaN              NaN                NaN   
std      865.881054        NaN       NaN              NaN                NaN   

       Gender   

# customer and transaction

In [20]:
#  Merge the datasets on 'Customer_ID'
merged_data = pd.merge(transaction_data, customer_data, on='Customer_ID', how='left')

#  Total transactions and total amount spent
total_transactions = merged_data['Transaction_ID'].nunique()
total_amount_spent = merged_data['Amount'].sum()
print(f"Total Transactions: {total_transactions}")
print(f"Total Amount Spent: {total_amount_spent}")

#  Segment analysis: transactions and amount per customer segment
segment_analysis = merged_data.groupby('Segment').agg(
    total_transactions=('Transaction_ID', 'nunique'),
    total_amount_spent=('Amount', 'sum')
).reset_index()
print("Segment Analysis:")
print(segment_analysis)

#  Customer with the highest total spend
customer_spend = merged_data.groupby(['Customer_ID', 'First_Name', 'Last_Name']).agg(
    total_spent=('Amount', 'sum')
).reset_index()
highest_spender = customer_spend.sort_values(by='total_spent', ascending=False).iloc[0]
print("Highest Spender:")
print(highest_spender)

Total Transactions: 2998
Total Amount Spent: 771384.9093952179
Segment Analysis:
   Segment  total_transactions  total_amount_spent
0  Premium                 927       240476.869785
1  Regular                1028       261961.349739
2      VIP                1043       268946.689871
Highest Spender:
Customer_ID           1495
First_Name           Emily
Last_Name             Reed
total_spent    1686.089981
Name: 958, dtype: object


In [21]:
# Age calculation and group analysis
customer_data['Birthdate'] = pd.to_datetime(customer_data['Birthdate'], errors='coerce')
current_year = datetime.now().year
customer_data['Age'] = current_year - customer_data['Birthdate'].dt.year

# Define age groups
bins = [0, 18, 30, 45, 60, 100]
labels = ['Under 18', '18-30', '31-45', '46-60', 'Above 60']
customer_data['Age_Group'] = pd.cut(customer_data['Age'], bins=bins, labels=labels, right=False)

# Merge age data with transaction data
merged_data_with_age = pd.merge(transaction_data, customer_data[['Customer_ID', 'Age_Group']], on='Customer_ID', how='left')

# Analyze spending per age group
age_group_analysis = merged_data_with_age.groupby('Age_Group').agg(
    total_transactions=('Transaction_ID', 'nunique'),
    total_amount_spent=('Amount', 'sum')
).reset_index()
print("Age Group Analysis:")
print(age_group_analysis)

Age Group Analysis:
  Age_Group  total_transactions  total_amount_spent
0  Under 18                   0            0.000000
1     18-30                 489       122141.209730
2     31-45                 611       157901.969951
3     46-60                 661       174245.989742
4  Above 60                1237       317095.739972


In [22]:
#  Analyze spending over time (monthly)
merged_data['Purchase_Date'] = pd.to_datetime(merged_data['Purchase_Date'], errors='coerce')
merged_data['Year_Month'] = merged_data['Purchase_Date'].dt.to_period('M')

# Spending analysis per month
monthly_spending_analysis = merged_data.groupby('Year_Month').agg(
    total_transactions=('Transaction_ID', 'nunique'),
    total_amount_spent=('Amount', 'sum')
).reset_index()
print("Monthly Spending Analysis:")
print(monthly_spending_analysis)

Monthly Spending Analysis:
   Year_Month  total_transactions  total_amount_spent
0     2019-09                  10         2049.580011
1     2019-10                  59        14544.049944
2     2019-11                  65        17697.310108
3     2019-12                  51        12623.670019
4     2020-01                  46        11957.809948
..        ...                 ...                 ...
56    2024-05                  53        15950.339988
57    2024-06                  45        13029.979948
58    2024-07                  49        10871.099924
59    2024-08                  44        10150.460047
60    2024-09                  42         9882.710029

[61 rows x 3 columns]


In [23]:
#  Customer loyalty: number of transactions and average spend per customer
loyalty_analysis = merged_data.groupby('Customer_ID').agg(
    total_transactions=('Transaction_ID', 'count'),
    total_amount_spent=('Amount', 'sum'),
    first_purchase=('Purchase_Date', 'min'),
    last_purchase=('Purchase_Date', 'max')
).reset_index()

loyalty_analysis['avg_amount_per_transaction'] = loyalty_analysis['total_amount_spent'] / loyalty_analysis['total_transactions']
print("Customer Loyalty Analysis:")
print(loyalty_analysis)

#  New vs Existing Customers Analysis
customer_data['Join_Date'] = pd.to_datetime(customer_data['Join_Date'], errors='coerce')

# Define 'new' customers as those who joined in the last year from the most recent transaction date
recent_cutoff = merged_data['Purchase_Date'].max() - pd.DateOffset(years=1)
customer_data['Customer_Status'] = customer_data['Join_Date'].apply(lambda x: 'New' if x >= recent_cutoff else 'Existing')

# Merge the customer status back into the transaction data
merged_data_with_status = pd.merge(merged_data, customer_data[['Customer_ID', 'Customer_Status']], on='Customer_ID', how='left')

Customer Loyalty Analysis:
      Customer_ID  total_transactions  total_amount_spent first_purchase  \
0               2                   2          742.369995     2021-05-02   
1               4                   1          336.010010     2021-01-19   
2               5                   2          627.360001     2022-06-22   
3               6                   1           65.389999     2020-07-01   
4               7                   1          162.899994     2019-10-21   
...           ...                 ...                 ...            ...   
1899         2994                   1          332.070007     2020-05-17   
1900         2996                   1          241.339996     2023-10-21   
1901         2997                   1           58.939999     2023-08-11   
1902         2998                   2          354.910004     2020-10-17   
1903         2999                   1          316.739990     2024-04-11   

     last_purchase  avg_amount_per_transaction  
0       202

In [24]:
# Analyze total spend and transactions by customer status
status_analysis = merged_data_with_status.groupby('Customer_Status').agg(
    total_transactions=('Transaction_ID', 'nunique'),
    total_amount_spent=('Amount', 'sum')
).reset_index()
print("Customer Status Analysis (New vs Existing):")
print(status_analysis)

#  Identify inactive customers (no transactions in the last 6 months)
inactive_cutoff = merged_data['Purchase_Date'].max() - pd.DateOffset(months=6)
inactive_customers = loyalty_analysis[loyalty_analysis['last_purchase'] < inactive_cutoff]
print("Inactive Customers:")
print(inactive_customers)

Customer Status Analysis (New vs Existing):
  Customer_Status  total_transactions  total_amount_spent
0        Existing                2414       622894.279368
1             New                 584       148490.630027
Inactive Customers:
      Customer_ID  total_transactions  total_amount_spent first_purchase  \
0               2                   2          742.369995     2021-05-02   
1               4                   1          336.010010     2021-01-19   
2               5                   2          627.360001     2022-06-22   
3               6                   1           65.389999     2020-07-01   
4               7                   1          162.899994     2019-10-21   
...           ...                 ...                 ...            ...   
1897         2991                   1          137.830002     2019-10-04   
1898         2992                   2          457.130005     2020-05-13   
1899         2994                   1          332.070007     2020-05-17   
19